In [1]:
import os 
os.chdir("../")
os.getcwd()

'e:\\L2RPN\\Dreamer_V3_Implimentation\\Dreamer-V3'

In [2]:
import grid2op
from grid2op.Reward import L2RPNSandBoxScore
from lightsim2grid import LightSimBackend
from dreamer.modules.worldModel import WorldModel
from dreamer.Utils.utils import Config
from dreamer.modules.actor_critic import ActorCritic
from dreamer.modules.dreamerTrainer import DreamerTrainer
from dreamer.Utils.utils import Config
import numpy as np

In [3]:
# Create environment
env = grid2op.make(
    "l2rpn_case14_sandbox",
    reward_class=L2RPNSandBoxScore,
    backend=LightSimBackend()
)

c:\Users\hrand\anaconda3\envs\MARL2023paper_env\Lib\site-packages\grid2op\Backend\backend.py:2062: UserWarning: The backend implementation you are using is probably too old to take advantage of the new feature added in grid2op 1.10.0: the possibility to have more than 2 busbars per substations (or not). To silence this warning, you can modify the `load_grid` implementation of your backend and either call:
- self.can_handle_more_than_2_busbar if the current implementation    can handle more than 2 busbsars OR
- self.cannot_handle_more_than_2_busbar if not.
And of course, ideally, if the current implementation of your backend cannot handle more than 2 busbars per substation, then change it :-)
Your backend will behave as if it did not support it.
  warnings.warn("The backend implementation you are using is probably too old to take advantage of the "


In [4]:
config = Config.from_yaml("config.yml")
actor_critic = ActorCritic(config=config)
world_model = WorldModel(config)
world_model.load_world_model()
trainer = DreamerTrainer(world_model=world_model, actor_critic=actor_critic,config=config)

Cuda is availabel for Training
Model loaded successfully from dreamer\models\recurrent_model
Model loaded successfully from dreamer\models\encoder
Model loaded successfully from dreamer\models\dynamic_predictor
Model loaded successfully from dreamer\models\reward_predictor
Model loaded successfully from dreamer\models\continue_predictor
Model loaded successfully from dreamer\models\decoder
World model loaded at dreamer\models


e:\L2RPN\Dreamer_V3_Implimentation\Dreamer-V3\dreamer\modules\recurrentModel.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path)

In [5]:
# Training parameters
num_epochs = 100
batch_size = 32
steps_per_epoch = 10
import torch
obs = env.reset()
obs = obs.to_vect()

In [6]:
obs = torch.tensor(obs)
obs.shape

torch.Size([467])

In [7]:
latent_states, hidden_states, actions, rewards, continues =trainer.imagine_trajectory(initial_state=obs)

torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])


In [8]:
print("Size of latent state: ", len(latent_states))
print("Size of hidden state: ", len(hidden_states))
print("Size of actions: ", len(actions))
print("Size of rewards: ", len(rewards))
print("Size of continues: ", len(continues))

Size of latent state:  16
Size of hidden state:  16
Size of actions:  16
Size of rewards:  16
Size of continues:  16


In [9]:
latent_states[0].detach()

tensor([[  467.8332, -4184.3657, -4731.0391, -1196.9907,  3243.4456, -2850.3789,
          3647.7180, -2546.7937,   844.2480,   928.1086,  2180.5015,  3202.4246,
          1610.8192,  3428.7324,  2118.3696, -1491.5612, -3737.6987,   319.6416,
          -675.1724, -3293.5249,  2917.8738,   145.6512, -2020.3873,  -124.2827,
          1920.9397,   327.3677,  4241.2739,  -206.0128,  -678.0339, -1960.9033,
          4373.1343,  3843.2397,  1646.9080,  1835.5894, -1349.0554,   295.2041,
           766.0688,  2011.6403,  2642.6890, -3158.5127,  3055.8572,  1497.2297,
         -2185.5283, -1372.6190,  3444.0374, -1188.3472,   231.3518, -1519.9739,
          3495.4817, -1137.0533, -1348.1296, -4903.4585,  -994.5637, -3184.1550,
         -2997.5564, -3103.1060,  2353.1155,  4944.1792,   510.8064, -2766.5117,
         -3421.8718, -3327.9368,  3700.2791, -4429.6387]], device='cuda:0')

In [10]:
# Stack all latent and hidden states from all timesteps
latent_stack = torch.stack(latent_states)    # Shape: [time_steps, batch_size, latent_dim]
hidden_stack = torch.stack(hidden_states)    # Shape: [time_steps, batch_size, hidden_dim]
            
# Concatenate latent and hidden states
states = torch.cat([latent_stack, hidden_stack], dim=-1) 
print(states.shape)

torch.Size([16, 1, 192])


In [11]:
trainer.initialize_buffer(num_trajectories=50)

torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
torc

In [12]:
batch = trainer.replay_buffer.get_sample(batch_size=32)
batch['states'].shape

torch.Size([32, 16, 1, 192])

In [14]:
initial_state = batch['states'][:, 0, :]  # [batch_size, obs_dim]

In [15]:
initial_state.shape

torch.Size([32, 1, 192])

In [13]:
ts = trainer.train_step(initial_state=obs, replay_buffer_batch=batch)

torch.Size([1, 128])
torch.Size([1, 467])
torch.Size([1, 128])
torch.Size([1, 64])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  torch.Size([1, 179])
shape of State:  torch.Size([1, 192])
shape of Actiion:  to